In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero
from statistics import mean

dir_ = './data/'

In [2]:
file_name = 'normalized_minmax_filter_track_5_user_100.csv'
pred_m = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test_m = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
all_user = pd.read_pickle(os.path.join(dir_, file_name[:-3] + 'pkl'))
pred_l = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test_l = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))

num_user = len(pred_m['uid'].unique())
num_user

220

In [3]:
pop_dict = {}
for i in all_user['tid'].unique():
    pop_dict[i] = len(all_user[all_user['tid']==i])/num_user

In [4]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [5]:
# Personal Recommendation
n=50
top_n_lists_m = []
top_n_lists_l = []

for i in tqdm(range(num_user)):
    pred = pred_m[pred_m['uid'] == i]
    pred = pred.sort_values(by=['rating'],  ascending=False)
    pred = pred[:n]
    top_n_list_m = []
    for _, row in pred.iterrows():
        top_n_list_m.append(row[1])
    top_n_lists_m.append(top_n_list_m)
    
    pred = pred_l[pred_m['uid'] == i]
    pred = pred.sort_values(by=['rating'],  ascending=False)
    pred = pred[:n]
    top_n_list_l = []
    for _, row in pred.iterrows():
        top_n_list_l.append(row[1])
    top_n_lists_l.append(top_n_list_l)

In [6]:
precision = []
recall = []
nDCG = []
HitNOV = []

test = test_m
hits = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists_m[i]
    
    hit = 0
    nov = 0
    truth_rating = []
    for j in top_n_list:
        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        if len(t) > 0:
            hit += 1
            nov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hits[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    HitNOV.append(nov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('HitNOV', mean(HitNOV))
print('HitCOV', list(hits).count(True)/len(all_user['tid'].unique()))


presicion:  0.11736363636363636
recall:  0.010816227286446883
nDCG:  0.05524507802492375
HitNOV 0.0723194214876033
HitCOV 0.006876603176335753


In [7]:
precision = []
recall = []
nDCG = []
HitNOV = []

test = test_l
hits = np.zeros(len(all_user['tid'].unique()), dtype=bool)
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists_l[i]
    
    hit = 0
    nov = 0
    truth_rating = []
    for j in top_n_list:
        t = test[test['uid'] == i]
        t = t[t['tid']==j]
        if len(t) > 0:
            hit += 1
            nov += (1-pop_dict[j])
            truth_rating.append(t.iloc[0]['rating']) # For nDCG
            hits[int(j)] = True
        else:
            truth_rating.append(0) # For nDCG
    max_rating = test[test['uid']==i].sort_values(by=['rating'],  ascending=False)['rating'].values[:n]

    precision.append(hit/n)
    recall.append(hit/len(test[test['uid'] == i]))
    nDCG.append(ndcg_at_k(truth_rating, max_rating, n, method=1))
    HitNOV.append(nov/n)

print('presicion: ', mean(precision))
print('recall: ', mean(recall))
print('nDCG: ', mean(nDCG))
print('HitNOV', mean(HitNOV))
print('HitCOV', list(hits).count(True)/len(all_user['tid'].unique()))


presicion:  0.11272727272727273
recall:  0.00880718344622947
nDCG:  0.0642232059469411
HitNOV 0.07217933884297521
HitCOV 0.009405301169750222
